<a href="https://colab.research.google.com/github/vatsal095/My-Contrct/blob/master/More_than_you_ever_wanted_to_know_about_power_perpetuals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Joe Clark and Andrew Leone - opyn research*

# Introduction and scope

This is a guide to the pricing and use of power perpetual contracts. 

The goal is to demonstrate the results from first principals but without undue formality. For the important results we illustrate with implementations in Python. 

The guide is relatively self-contained, but we
include some useful mathematical background in the appendix.

## Introducing the contracts

**Everlasting options** are contracts with no expiry that transfer regular funding between long and short sides sides based on the prevailing (mark) price relative to the target (index) price. This mechanism allows any traditional derivative to be traded in a perpetual form (without expiries). 

The contract will behave approximately like a portfolio of rolled fixed expiry contracts, where each expiring contract is replaced with a later expiry. The funding provides incentives for the price to track the index.

**Power futures** are derivatives that pay a power of an asset price. For example, a 2-future pays the square of the asset price at expiry. Power futures usefully generalize ordinary futures ($price^1$) and bonds ( $price^0$), and can be priced under Black-Scholes-Merton or more general assumptions on price dynamics.

**Power perpetuals** are power futures implemented in perpetual form. From a pricing perspective, these are a series of fixed expiry power futures priced at each funding period. In the simple case of flat volatilty with continuous funding, the price of the power perpetual will approach the price of a fixed expiry contract expiring at the funding period.

## Scope of this guide


*Power contracts:*
- Derive the formula for any power contract under log-normal prices.

*Everlasting options:*
- Derive the pricing formula for everlasting options using an arbitrage portfolio of the underlying contract at different expiries.

*Power perpetuals*
- Derive the pricing formula for power perpetuals by combining the fixed expiry power contracts the with everlasting options pricing formula.
- Show the special case of log-normal price with constant volatility is approximated by the fixed expiry contract.

*Replication*
- Show the exact replicating portfolio for power perpetuals in terms of combinations of bonds, futures, and options. This gives a useful model-free way to price power perpetuals if we have access to option prices.
- Demonstrate the replication.

*Applications*
- Show how any function can be replicated by a suitable combination of powerperpetuals.
- Show an example of hedging a call option with a quadratic power perpetual.


# Power contracts

We want to price a claim that pays the $p$'th power of a non-yielding asset price at expiry: 

$$D(T,T) = S_T^p $$

where $D(t,T)$ as the price of a claim at $t$ expirying at $T$, $S_T$ is the value of the asset at $T$, and $p$ any real number. The valuation is the same basic process we use to price options under Black Scholes Merton -- we choose a distibution for the price of the underlying asset and price the derivative under the risk neutral version of that distribution. This gives a convenient closed form with lognormal prices and can be generalized to any price process.  


## Pricing with respect to a numeraire

The martingale condition gives us that the price of a derivative $D(T,T)$ divided by some numeraire asset with value $N_t$ is

$$E_*\left(\frac{D(T,T)}{N_T} |S_t\right) = \frac{D(t,T)}{N_t}$$

This says that the ratio of assets has no drift under the risk neutral distribution. This guarantees that pricing is arbitrage free.

## Log normal risk neutral

Assume some asset has value $S_t$ at time $t$ and that for some future time $T$. 

$$\log S_T|S_t \sim N(\log S_t + \gamma \tau, \sigma^2 \tau)$$

Where $\tau = T-t$.

The martingale condition for the price of a money market account $M_t$ starting at 1 and growing to $e^{r\tau}$ at $T$ is 

$$E_*\left(\frac{M_T}{S_T} |S_t\right) = \frac{1}{S_t}$$

Evaluating the left hand side directly under the log normal is 

$$E_*\left(\frac{M_T}{S_T}\right) = e^{- \ln S_t - \gamma \tau + \frac{1}{2}\sigma^2 \tau + r \tau}$$ 

For the martingale condition to hold we must have $\gamma = r+0.5\sigma^2$

So the distribution of $\log S_T$ under the risk neutral measure $*$ with respect to the asset numeraire is

$$\log S_T|S_t \sim_* N(\log S_t + (r+0.5\sigma^2) \tau, \sigma^2 \tau)$$


## Pricing power futures under the risk neutral distribution with respect to the asset

First we price the **power forward** with payout $S_T^p$

Under the fundamental theorem 

$$E_*\left(\frac{D(T,T)}{N_T} |S_t\right) = \frac{D(t,T)}{N_t}$$

It's convenient to use the asset numeraire for $D_p(T,T) = S_T^p$ which is 

$$E_*\left(\frac{S_T^p}{S_T} |S_t\right) = \frac{D(t,T)}{S_t}$$

So the price of a power forward at time $t$ is 

$$D(t,T) = S_t E_*(S_T^{p-1}|S_t)$$

We can use the risk neutral distribution from the previous section combined with the fact that for a $X\sim Lognormal(\mu,\sigma^2)$

$$E(X^k) = e^{k\mu + \frac{k^2}{2}\sigma^2}$$

Combing with our results on the mean and variance of our log-normal price, we have the price of a power forward with expiry 

$$D^p(t,T) = S_t^p e^{(p-1)(r+ \frac{p\sigma^2}{2})\tau}$$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
def powerFwd(S, r, q, T, sigma, p):
  '''
  Power forward with
  Inputs:  
  S - current underlying price
  r - risk free rate (numeraire yield)
  q - asset yield
  T - time to expiry
  p - power

  Returns:
  F - power forward price
  '''
  #return S**p * np.exp((p-1)*(r+(p/2)*sigma**2)*T) # (no yield version)
  return S**p * np.exp( ((p-1)*(r+(p*sigma**2)/2) -p*q)*T)

In [ ]:
#@title Power forwards interactive demonstration { run: "auto", vertical-output: true }

%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set()

# Fixed params
r = 0 
q = 0

def f(T, sigma, p):
    plt.figure(2)
    x = np.linspace(0, 100, num=20)
    plt.plot(x, [powerFwd(x_, r, q, 0, sigma, p) for x_ in x])
    plt.plot(x, [powerFwd(x_, r, q, T, sigma, p) for x_ in x])
    plt.legend(['Index', 'Price'])
    plt.xlabel('Underlying price')
    plt.ylabel('Price')
    plt.autoscale()
    plt.show()

interactive_plot = interactive(f, T= (0, 1, 0.1), sigma = (0.05,1.5,0.1), p=(0,5,0.5), continuous_update=True)
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot


interactive(children=(FloatSlider(value=0.0, description='T', max=1.0), FloatSlider(value=0.7500000000000001, …

# Everlasting options

Everlasting options (see [3] and [4]) are a perpetual derivative with a regular funding payment. The funding payment is determined by the difference between the mark price of the perpetual and the expiry payout for underlying derivative. 

Funding in period $t$ is

$d_t = M_\infty(t) - V(t,t)$

Where $M_\infty(t)$ is the prevailing (mark) price and $V(t,t)$ is the value of a derivative paying the index value at expiry. 

For example, a call option with strike $K$ the funding payment is 

$d_t = M_\infty(t) - C_K(t,t) =  M_\infty(t) -  (S_t - K)^+$

For a future (a zero strike call) the funding payment is the difference between the current futures price and the spot price

$d_t = M_\infty(t) - S_t$

For a derivative paying $S_t^2$ the payment is

$d_t =  M_\infty(t) -  S_t^2$



### Valuation with fractional funding

The value of the perpetual paying the underlying derivative payoff $V(t,t)$ at time $t$ and receiving $\frac{d_{t_i}}{N}$ for each time period $t_i = \frac{i}{N} $ is

$V_{\infty}(t) = \sum_{i=1}^\infty  a_i V(t, t_i)$

Where 

$a_i = \frac{1}{N} \left( \frac{N}{N+1} \right)^i$

This can be shown with an aribitrage argument. If $M_\infty(t_0)< V_{\infty}(t_0)$ we define a trading strategy for some small time  $\epsilon$:

At $t_0$:
- Long 1 perpetual at $t_0$
- Short $a_i $ of the underlying derivative expiring at each $t_i$

At $t_i-\epsilon$:
- Sell $a_i$ of the perpetual

This portfolio has value $V_{\infty}-M_\infty(t_0)>0$ and at each funding time. At each $t_i$ the cashflow is 


$\underbrace{ a_i M_\infty(t_i)}_{\mbox{perp sale proceeds} } + \underbrace{ \frac{1}{N} (1- \sum_{j=1}^i a_j) (M_\infty(t_i)- V(t_i,t_i) )}_{\mbox{funding payment} }-\underbrace{ a_i V(t_i,t_i)}_{\mbox{underlying derivative payoff} } $

Since  $\frac{1}{N} (1- \sum_{j=1}^i a_j) =a_i$

$\underbrace{ a_i M_\infty(t_i)}_{\mbox{perp sale proceeds} } + \underbrace{ a_i(M_\infty(t_i)- V(t_i,t_i) )}_{\mbox{funding payment} }-\underbrace{ a_i V(t_i,t_i)}_{\mbox{underlying derivative payoff} } =0 $

Similarly if $M_\infty(t_0)> V_{\infty}(t_0)$ the arbitrage portfolio buys the options and sells the perpetual.


In [ ]:
#@title Everlasting option weights demonstration { run: "auto", vertical-output: true }

%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set()

def f(N):
    plt.figure(2)
    i = [i_ for i_ in range(1,N*5)]
    wts = [1/N  * (N/(N+1))**i_ for i_ in i]
    plt.scatter(i, wts)
    plt.xlabel('Funding period')
    plt.ylabel('Weight')
    plt.legend(['Weight'])
    plt.autoscale()
    plt.show()

interactive_plot = interactive(f, N= (1, 365,1), continuous_update=True)
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot


interactive(children=(IntSlider(value=183, description='N', max=365, min=1), Output(layout=Layout(height='350p…

# Power perpetuals

The price of a power future on an asset with continuous divideends (see Haug 1997)

$V^p(t,T) = S_t^p e^{((p-1)(r+ \frac{p\sigma^2}{2}) -pq )(T-t)} = S_t^p e^{k\tau}$ 

Where $\tau = T-t$. If we have a perpetual that divides funding across $N$ periods we can price

$V_\infty(t) = \sum_{i=1}^\infty a_i S_{t}^p e^{\frac{ki}{N} }$

If volatility and rates are constant this series simplifies to 

$V_\infty(t) = S_t^p\left( \frac{1}{(N+1) \exp \left( \frac{-k}{N}\right) -N  }  \right)$

(since $\sum_{i=1}^{\infty} a^i = \frac{1}{1-a}-a^0$ for $a<1$)


## Power perpetuals with frequent compounding

As we increase the number of periods $N$ relative the carry $k$, the power perpetual price is close to the fixed expiry power future expiring at the funding period. This is intuitive since funding payments on a power perpetual should have the same impact as the carry a fixed expiry future. 

The ratio between the expiring power futures and the perpetual is. 

$\frac{$V^p(t,T)}{V_\infty(t) } = e^{k}\left((N+1) e^{-k/N} -N\right)$

This makes the fixed expiry future a good approximation for a perpetual with continuous funding for plausible ranges of volatility.  

For example if the annual volatilty is 80\% and dividends are paid each minute over the course of a day, the 2-perpetual is within 0.003\% of the one day fixed expiry price. This means that calculating volatilty for a high frequency rebalancing perpetual using fixed expiry will be accurate to within 0.02 volatilty points.

For longer funding periods and higher volatililty this is less precise, though it is a useful rule of thumb. 

In [ ]:
def powerPerpetual(S, r, q, N, sigma, p):
  '''
  Power perpetual price.
  Inputs:  
  S - current underlying price
  r - risk free rate (numeraire yield) for N periods
  q - asset yield N periods
  N - number of funding periods
  sigma - volatility per N funding periods
  p - power

  Returns:
  P - power perpetual price
  '''
  k = ((p-1)*(r+(p*sigma**2)/2) -p*q)
  #a = N/(N+1) * np.exp(k / N)
  #return  (S**p /N ) * ( 1/ ( 1 - a) -1)  

  return  S**p * 1/((N+1)*np.exp(-k / N) - N)

In [ ]:
#@title **Power perpetuals interactive demonstration**  { run: "auto", vertical-output: true }

%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set()

#@markdown ---
#@markdown ### Funding paid over (days):
D =  30# @param {type: "number"}
#@markdown ### Underlying price:
S0 = 100 # @param {type: "number"}
#@markdown ### Risk free rate:
r = 0 # @param {type: "number"}
#@markdown ### Yield:
q = 0 # @param {type: "number"}
#@markdown ### Power:
p = 2 # @param {type: "number"}
#@markdown ### Number of funding periods:
N =  1000# @param {type: "number"}
#@markdown ### Volatility:
sigma =  0.92# @param {type: "number"}
sigma_ = sigma * np.sqrt(D/365) # Volatility per funding period
#@markdown ---
#@markdown ### The power perpetual is close to a fixed expiry with more frequent funding
fixedExp = powerFwd(S0, r, q, 0, sigma_, p)
pFwd = powerFwd(S0, r, q, 1, sigma_, p)
pPerp = powerPerpetual(S0, r, q, N, sigma_, p)
print('Index: ' + str(fixedExp))
print('Fixed expiry mark: ' + str(pFwd))
print('Power perpetual mark: ' + str(pPerp))
print('Difference between fixed expiry and power perp: ' + str(round(np.log(pFwd/pPerp)*100,3 )) + '%')


Index: 10000.0
Fixed expiry mark: 10688.41490194988
Power perpetual mark: 10713.976151200273
Difference between fixed expiry and power perp: -0.239%


# Replication

## Replicating portfolios for power perpetuals


Any function $g(x)$ can be decomposed as (see Carr and Madan 2001)

$g(x) = g(A) + g'(A)(x-A)  + \int_0^A (K-x)^+ g''(x)dK +\int_A^\infty (x-K)^+ g''(x)dK$

This is 

- A zero coupon bond with face value $g(A)$
- $g'(A)$ notional of the forward
- $g''(K)dK$ notional of puts between $0$ and $A$ and 
- $g''(K)dK$ notional of calls between $A$ and $\infty$ and 

This can sometimes be usefully expressed purely in terms of calls by setting $A=0$, then

$g(x)= g(0) + g'(0)x + \int_0^\infty (x-K)^+ g''(x)dK$


## The replicating portfolio of a power future

The replicating portfolio for a power contract paying $g(S_T) = S_T^p$ will depend on $p$. We consider 4 cases.

We'll refer to the p-forward expiring at $T$ as $F^p$ and the zero coupon bond as $Z(t,T)$

### 1. Bond case: $p=0$

- A zero coupon bond with face value $\$1$

The total cost is the zero coupon bond price 

$Z(t,T) = e^{-r(t-T)}$

### 2. Linear case: $p=1$

- $F^1$ notional of the bond
- 1 unit of the forward paying $(S_T-F^1)$

The power forward is:

$F^1 = S(t)^1 e^{-q(T-t)}$

### 3. Below quadratic non-linear: $(p<2) \cap (p \notin \{0,1\}) $ 

- A zero coupon bond with face value $F^p$
- $p A ^{p-1}$ notional of the forward
- $p(p-1)K^{p-2} dK$ notional of puts between $0$ and $A$ and 
- $p(p-1)K^{p-2} dK$ notional of calls between $A$ and $\infty$ and 


### 4. At least quadaratic: $p\geq 2$ 
- $p(p-1)K^{p-2} dK$ notional of calls at all strikes

For $p=2$ we have the special case where a constant $2dK$ call options replicates the quadratic power future.

# Example: A 2-perp with daily funding


The replicating portfolio for a 2-perp paying funding across $N=365$ periods is 

$\omega(i,K) = \frac{1}{N} (\frac{N}{N+1})^i 2 d K$

For a real-world replication this can be approximated with 

$\tilde{\omega}(i,K) = \frac{1}{N} (\frac{N}{N+1})^i 2 \Delta K \Delta i$

If we use $\Delta K=1$, $\Delta i = 30$ this is 

$\tilde{\omega}(i,K) = \frac{60}{365} (\frac{365}{366})^i$ for $i = 1,30,60,\ldots$ 

So this would be 0.151 notional to each $1 of strike spacing for one month calls, 0.139 for 2 month calls, and so forth.


In [ ]:
# Span functions for Carr Madan (2001)

def callSpan(g, dg, dsqg, dK=0.1, maxStrike= 100):
    '''
    Span with bonds, calls, futures
    g(x) = g
    g'(x) = dg
    g''(x) = dsqg

    Replicate a function with
    g(S_T) = g(0) + g'(0)S_T + \int_0^\infty (x-K)^+ g''(K)dK

    '''
    
    eps = np.finfo(float).eps # use small number to avoid div 0 
    strikes = np.arange(dK, maxStrike, step=dK)
    return lambda x : g(eps) + dg(eps)*x + np.sum([dsqg(K)*np.maximum(x-K,0)*dK for K in strikes ])

def optionSpan(g,dg, dsqg, A, dK=0.1, maxStrike= 100):
    '''
    Span with bonds, puts, calls, futures
    g(x) = g
    g'(x) = dg
    g''(x) = dsqg

    Replicate a function with
    g(S_T) = g(A) + g'(A)(S_T-A) + \int_0^A (K-x)^+ g''(K)dK + \int_A^\infty (x-K)^+ g''(K)dK

    '''
    strikes = np.arange(dK, maxStrike, step=dK)
    putStrikes = strikes[strikes<A]
    callStrikes = strikes[strikes>=A]
    return lambda x : g(A) + dg(A)*(x-A) + np.sum([dsqg(K)*np.maximum(K-x,0)*dK for K in putStrikes]) \
                                     + np.sum([dsqg(K)*np.maximum(x-K,0)*dK for K in callStrikes])

The `optionSpan` function can be used to price any well behaved function of the price. The `callSpan` function is a special case of `optionSpan` that we can use provided that $g(0)$ and $g'(0)$ are defined. 

In [ ]:
#@title Power forwards replication demonstration (p=2) { run: "auto", vertical-output: true }

%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set()

def f(dK):
    maxStrike = 200
    plt.figure(2)
    x = np.linspace(0, 100, num=20)
    g = lambda x: x**2
    g1 = lambda x: 2*x
    g2 = lambda x: 2

    # Just calls
    spanFun = callSpan(g, g1, g2, dK=dK, maxStrike=maxStrike )
    # Puts and calls
    spanFunCP = optionSpan(g, g1, g2, 50, dK=dK, maxStrike=maxStrike)
    
    # Plot target function vs replication
    priceRange = np.arange(1,maxStrike) 
    sns.set()
    plt.plot(priceRange, [spanFun(x) for x in priceRange])
    plt.plot(priceRange, [spanFunCP(x) for x in priceRange])
    plt.plot(priceRange, [x**2 for x in priceRange])
    plt.legend(['Call replication','Call and put replication', 'Target payoff'])
    plt.autoscale()
    plt.show()

interactive_plot = interactive(f, dK= (1, 10, 1), continuous_update=True)
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(IntSlider(value=5, description='dK', max=10, min=1), Output(layout=Layout(height='350px'…

The replication across for a power perpetual is of the power future expiring at each funding period. 

In [ ]:
from scipy.stats import norm
def powerOption(S, K, r, q, T, sigma, p, isCall=True):
  '''
    Power option with
    Inputs:  
    S - current underlying price
    K - strike
    r - risk free rate (numeraire yield)
    q - asset yield 
    T - time to expiry
    p - power
    isCall - True for call, False for out

    Returns:
    P - power option price
  '''

  if K==0:
    K = 1e-10

  d1 = (np.log((S)/(K**(1/p))) +(r-q+(p-0.5)*sigma**2)*T)/(sigma*np.sqrt(T))
  d2 = d1 - p*sigma*np.sqrt(T)
  # Fwd
  F = powerFwd(S,r,q,T,sigma, p)
  # ZCB
  Z = np.exp(-r*T)
  # Call and put prices
  callPrice = F * norm.cdf(d1) - K*Z * norm.cdf(d2)
  putPrice = K*Z * norm.cdf(-d2) - F * norm.cdf(-d1)
  if isCall:
    return callPrice
  else:
    return putPrice

# BSM pricer
bsmPrice = lambda S, K, r, q, T, sigma, isCall: powerOption(S, K, r, q, T, sigma, 1, isCall)

def volSurface(K, T):
  # Dummy flat vol surface fn
  return sigma

def stripCost(T, dK, maxStrike =200):
  # Replicate 2*dK strip at expiry T  
  strikes = np.arange(0, maxStrike, dK)
  optPrices = [bsmPrice(S0, K, r, q, T, volSurface(K,T), True ) for K in strikes]
  # Sum option prices
  return 2*dK*np.sum(optPrices)

def stripVol(T, dK):
  repCost = stripCost(T, dK)
  return (np.log(repCost/S0**2)/T)**0.5 # (zero carry)

The power forward for $p=2$ is a strip of $2dK$ call options at each strike with strike width $dK$. The appoximation is more precise for small $dK$

In [ ]:
 #@title **Power forward replication demonstration (p=2, flat volatility surface)**  { run: "auto", vertical-output: true }

%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set()

#@markdown ---
#@markdown ### Expiry (days):
D =  17.5# @param {type: "number"}
#@markdown ### Underlying price:
S0 = 100 # @param {type: "number"}
#@markdown ### Risk free rate:
r = 0 # @param {type: "number"}
#@markdown ### Yield:
q = 0 # @param {type: "number"}
#@markdown ### Volatility:
sigma =  0.9# @param {type: "number"}
#@markdown ### Strike spacing:
dK =  0.1# @param {type: "number"}  


print('Replication cost: ' + str(stripCost(D/365, dK)))
print('Power forward: ' + str(powerFwd(S0,r,q,D/365,sigma,2)))
print('Implied volatility: ' + str(stripVol(D/365,dK)))

Replication cost: 10405.966811855678
Power forward: 10395.995765223093
Implied volatility: 0.9110406105910691


In [ ]:
 #@title **Power perpetuals replication demonstration (p=2, flat volatility surface)**  { run: "auto", vertical-output: true }

%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set()

#@markdown ---
#@markdown ### Funding paid over (days):
D =  17.5# @param {type: "number"}
#@markdown ### Underlying price:
S0 = 100 # @param {type: "number"}
#@markdown ### Risk free rate:
r = 0 # @param {type: "number"}
#@markdown ### Yield:
q = 0 # @param {type: "number"}
#@markdown ### Volatility:
sigma =  0.8# @param {type: "number"}
#@markdown ### Number of funding periods:
N =  200# @param {type: "number"}  
isCall = True

def volTerm(T):
  # dummy flat volatility term structucture
  return sigma

i = [i_ for i_ in np.arange(1, 10000)] #  
stripWt = lambda i:  1/N * (N/(N+1))**i
#pFwd = lambda i:  S0**2 * np.exp( sigma **2 * i/365)
pFwd = lambda i: powerFwd(S=S0, r=r, q=q, T=i/N, sigma=volTerm(i/N)* np.sqrt(D/365) ,p=2)
replicationCost = [stripWt(i_) * pFwd(i_) for i_ in i]
repCost = np.sum(replicationCost)
# Solve for vol at one year (zero carry)
impliedVol = ((np.log(repCost/S0**2))/(D/365))**0.5

print('Replication cost: ' + str(repCost))
print('Implied volatility: ' + str(impliedVol))

Replication cost: 10318.171024762312
Implied volatility: 0.8082545621144305


# Applications

## Taylor series replication of payoff functions

Taylor series represent a function as a series of weigths to polynomial components. This suggests a practical strategy to replicate any function as a combination of power perpetuals.

A Taylor series of a function of and underlying price $f(S_t)$ and some price movement $\Delta S$ is

$f(S_t) = \sum_{n=0}^\infty \frac{f^{(n)}(S_t)}{n!} (\Delta S)^n$

The payoff of an n-perpetual from the same price movement is (exclucluding funding)

$(S_t+\Delta S)^n - S_t^n $

We can isolate exposures to $(\Delta S)^n$ from a n-th order power perpetuals by hedging the remaining components with lower order perpetuals.

## Hedging options with 2-perpetuals

For example if $f(S_t)$ is a call option then the change in value due to the price change is

$f(S_t+\Delta S_t) - f(S_t) = \sum_{n=1}^\infty \frac{f^{(n)}(S_t)}{n!} (\Delta S)^n$. 

The payoff can be replicated by by holding the option's delta exposure to the price change, half the gamma to squared price changes, etc.

$f(S_t+\Delta S_t) - f(S_t) = \frac{\partial f}{\partial S} \Delta S +  \frac{1}{2}\frac{\partial^2 f}{(\partial S)^2} (\Delta S)^2 + O((\Delta S)^3)$

A 2-perpetual has a payoff (excluding funding) 

$(S_t+\Delta S)^2 - S_t^2 = 2 S_t \Delta S + (\Delta S)^2$

To hedge the second order movements (gamma), the notional size for the perpetual is

 $-\frac{1}{2}\frac{\partial^2 f}{(\partial S)^2}$ 


To hedge the first order movements (delta) we need to hedge the option delta and the delta from the 2-perpeual. The notional size of the 1-perpetual is

$- ( \frac{1}{2}\frac{\partial^2 f}{(\partial S)^2} 2 S_t + \frac{\partial f}{\partial S})$ 

We are still left with higher order $O((\Delta S)^3)$ terms, but the hedge holds remarkably well in practice.



# Appendix

## Useful mathematical background

To get to the pricing result for power forwards we need a result concerning the expected value of powers of a log-normal random variable. This involves a couple of tricks.

## Functions and moments of rvs

$f(x)$ is a pdf and $g(x)$ is some continuous fn

$E(g(x)) = \int_{-\infty}^\infty g(x)f(x)dx$

For example,

$E(x) = \int_{-\infty}^\infty xf(x)dx$

and 

$E(x^2) = \int_{-\infty}^\infty x^2 f(x)dx$

### Moment generating functions

A moment generating function for a random vairable $X$ is 

$M_X(t) = E(e^{tx})$

This is useful because if we know what it is we can figure out the moments $E(X^k)$

Note that the series expansion of $e^{tx}$ is


$e^{tx} = 1 + tX + \frac{t^2x^2}{2!} + \frac{t^3x^2}{3!} + \ldots$

Therefore 

$M_X(t) = E(e^{tx}) = 1 + tX + \frac{t^2 E(x^2)}{2!} + \frac{t^3E(x^3)}{3!} + \ldots$

Differentiating $n$ times wrt $t$ and evaluating at $t=0$ gives the $n$th moment $E(X^n)$

For example $n=2$

$\frac{ \partial M_X^2(t) }{\partial t^2} = E(x^2) + \frac{3*2*tE(x^2)}{3!} + \ldots = E(x^2)$

### Moments of the log normal

A log normal $x$ has a distribution 

$f(x) = \frac{1}{\sqrt{2 \pi} \sigma} \frac{e^{-(\ln x-\mu)^2 / 2\sigma^2}}{x}$

The moments $E(x^k)$ are

$E(x^k) = \int_0^\infty x^k f(x) dx $

$ = \int_0^\infty x^k \frac{1}{\sqrt{2 \pi} \sigma} \frac{e^{-(\ln(x)-\mu)^2 / 2\sigma^2}}{x} dx $ 

$ = \int_0^\infty x^{k-1} \frac{1}{\sqrt{2 \pi} \sigma} e^{-(\ln(x)-\mu)^2 / 2\sigma^2} dx $ 

Change of variable

$y = \ln x; x = e^y, dx = e^y dy$


$ = \int_{-\infty}^\infty e^{(k-1)y} \frac{1}{\sqrt{2 \pi} \sigma} e^{-(y-\mu)^2 / 2\sigma^2} e^y dy $ 

The middle bit is just a normal distibution for $y$ so the whole thing is

$E(e^{ky})$

for a normally distributed $y$


The mgf of the normal distribution is 

$M_y(t) = E(e^{ty})= e^{\mu t + \frac{1}{2}\sigma^2 t^2}$


So the moments of the lognormal are just the mgf of the normal.

# References

[1] Peter Carr and Dilip Madan, (2001) Towards a theory of volatility trading, Option Pricing, Interest Rates and Risk Management, Handbooks in Mathematical Finance (2001), 458–476* http://pricing.online.fr/docs/TradingVolatilityStrat.pdf

[2] Espen Haug. (1997) The Complete Guide To Option Pricing Formulas, McGraw-Hill New York.

[3] Dave White and Sam Bankman-Fried, (2021) Everlasting Options https://www.paradigm.xyz/papers/everlasting_options.pdf

[4] Dave White, Dan Robinson, Zubin Koticha, Andrew Leone, Alexis Gauba, and Aparna Krishnan, (2021) Power Perpetuals. https://www.paradigm.xyz/2021/08/power-perpetuals/



